# Examples for MuJoCo and Bimanual Sim Usage

In [1]:
%load_ext autoreload
%autoreload 2

### Low-level MuJoCo Model Instantiation

In [ ]:
import os
import mujoco
import mujoco.viewer

from robot_descriptions import aloha_mj_description

# instantiate the MuJoCo bimanual simulation model.
from xml.etree import ElementTree as ET

model = mujoco.MjModel.from_xml_path(aloha_mj_description.MJCF_PATH)
data = mujoco.MjData(model)

# launch an interactive viewer.
mujoco.viewer.launch(model, data)

In [ ]:
data.qpos

array([-1.24866767e-02, -9.85510335e-02,  1.00000000e+00,  8.35067762e-02,
        1.49315150e-01, -7.94138101e-02,  3.74128540e-02,  4.02052328e-02,
       -8.73947428e-04,  1.06232035e-01, -1.28021856e-01, -1.63744567e-02,
       -4.58504852e-02, -8.15300697e-02,  3.46516773e-02,  3.86005880e-02])

In [ ]:
data.ctrl

array([ 0.1571  , -0.591665,  0.039415,  0.34562 ,  0.819155, -0.12568 ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ])

In [ ]:
from pathlib import Path
from sim import BimanualSim

sim = BimanualSim(merge_xml_files=[Path('block.xml')])
sim.launch_viewer()

## BimanualSim Policy Deployment

In [3]:
import os
import numpy as np
import cv2
from IPython.display import Video
from sim import BimanualObs, BimanualSim

image_dims = (480, 640)  # h x w


def toy_policy(obs: BimanualObs) -> np.ndarray:
  # simply set the target to the current joint positions + 0.01
  return obs.qpos[list(range(7)) + list(range(8, 15))] + 0.01

# rollout simulation for 30 steps
sim = BimanualSim(camera_dims=image_dims, obs_camera_names=['wrist_cam_left'])
left_wrist_frames = []

obs = sim.get_obs()
for sim_step in range(30):
  left_wrist_frames.append(obs.visual[0])
  action = toy_policy(obs)
  obs = sim.step(action)

# save frames to video
os.makedirs('out', exist_ok=True)
video_path = 'out/example-bimanual-rollout.mp4'
video_writer = cv2.VideoWriter(video_path, cv2.VideoWriter.fourcc(*'H264'), 20, tuple(reversed(image_dims)))
for frame in left_wrist_frames:
  video_writer.write(cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_RGB2BGR))
video_writer.release()
Video(video_path, width=image_dims[1], height=image_dims[0])